In [ ]:
import os
import gc
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from sklearn.model_selection import KFold
from transformers import AutoTokenizer, AutoModel, AutoConfig, LongformerTokenizer

In [ ]:
device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu' )

In [ ]:
class config:
    batch_size = 4
    acc_steps = 8
    max_len = 1024
    lr = 2e-5
    weight_decay=1e-3

In [ ]:
label2segment={
    0: 'Lead',
    1: 'Lead',
    
    2: 'Position',
    3: 'Position',
    
    4: 'Evidence',
    5: 'Evidence',
    
    6: 'Claim',
    7: 'Claim',
    
    8: 'Concluding Statement',
    9: 'Concluding Statement',
    
    10: 'Counterclaim',
    11: 'Counterclaim',
    
    12: 'Rebuttal',
    13: 'Rebuttal'
}

# model

In [ ]:
class FeedbackModel(nn.Module):
    def __init__(self, num_labels):
        super(FeedbackModel, self).__init__()
        modelconfig = AutoConfig.from_pretrained(config.model_name)

        self.backbone = AutoModel.from_pretrained(config.model_name)
        self.output = nn.Linear(modelconfig.hidden_size, num_labels)
    
    def forward(self, input_ids, attn_mask, rpercentile):
        attn_outputs = self.backbone(input_ids, attn_mask)
        y=self.output(attn_outputs.last_hidden_state)
        return y

# dataset

In [ ]:
class FeedbackDataset( torch.utils.data.Dataset ):
    def __init__(self, df, tokenizer):
        self.tokenizer=tokenizer
        df=df.copy()
        self.content = df.content.values
    
    def get_tokenized_inputs(self, essay):
        tokenized_inputs = self.tokenizer(essay, is_split_into_words=True)
        word_ids = tokenized_inputs.word_ids()
        return (tokenized_inputs, word_ids)
    
    
    def __getitem__(self, idx):
        essay  = self.content[idx]
        (tokenized_inputs, word_ids) = self.get_tokenized_inputs(essay)
        word_ids[0] = -100
        word_ids[-1] = -100
        
        input_ids = tokenized_inputs['input_ids'][:config.max_len]
        attn_mask = tokenized_inputs['attention_mask'][:config.max_len]
        word_ids = word_ids[:config.max_len]
        seq_len = len(input_ids)
        
        if seq_len < config.max_len:
            len_diff = config.max_len - seq_len
            attn_mask += [0] * len_diff
            input_ids += [self.tokenizer.pad_token_id] * len_diff
            word_ids += [-100] * len_diff
        
        rpercentile = ((1 + np.arange(0, config.max_len))/seq_len) - 0.5        
        input_ids=torch.tensor(input_ids, dtype=torch.long)
        attn_mask = torch.tensor(attn_mask, dtype=torch.long)
        seq_len = torch.tensor(seq_len, dtype=torch.long)
        word_ids= torch.tensor(word_ids, dtype=torch.long)
        rpercentile = torch.tensor(rpercentile, dtype=torch.float32)
        
        return {
            'input_ids': input_ids,
            'attn_mask': attn_mask,
            'word_ids': word_ids,
            'seq_len': seq_len,
            'rpercentile': rpercentile
        }
    def __len__(self):
        return len(self.content)

# load models

In [ ]:
tokenizer=AutoTokenizer.from_pretrained('../input/longformer-base-tokenizer/longformer_base_tokenizer')
model = torch.load('../input/longformer-baseline-model/model.pt', map_location = device)

In [ ]:
def read_essay(filename):
    essay_folder='../input/feedback-prize-2021/test'
    filepath = os.path.join(essay_folder, filename)
    essay = ''
    with open(filepath) as file:
        essay = file.read()
    essay=essay.split()
    return essay

In [ ]:
test_files = os.listdir('../input/feedback-prize-2021/test')
test_df = []
for filename in test_files:
    test_df.append({
        'id': filename.replace(".txt", ''),
        'content': read_essay(filename)
    })
test_df = pd.DataFrame.from_dict(test_df)

In [ ]:
val_dataset   = FeedbackDataset(test_df, tokenizer)
val_dataloader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=2,
                                             shuffle=False,
                                             drop_last=False
                                            )

In [ ]:
def postprocess( y, word_ids):
    seq_len = len(y)
    prv_word_id=None
    predSegment=[]
    predTokens=[]
    
    preds=[]
    for i in range(seq_len):
        word_id = word_ids[i]
        if  (word_id== -100) or (prv_word_id == word_id):
            continue
        prv_word_id = word_id
        
        if y[i] not in label2segment:
            continue
        
        segment = label2segment[ y[i] ]
        predSegment.append(segment)
        predTokens.append( word_id )
    
    if len(predSegment) == 0:
        return []
    
    if len(predSegment) == 1:
        preds.append({
            'segment': predSegment[0],
            'word_ids': [predTokens[0]]
        })
        return preds
    else:
        num_tokens=len(predTokens)
        prv_id=0
        cur_id=0
        prv_segment=predSegment[0]
        
        for i in range(1, num_tokens+1):
            cur_id=i
            if (i!=num_tokens) and \
                (predTokens[i] == 1+predTokens[i-1]) and \
                (predSegment[i] == predSegment[i-1]):
                continue
            
            pred_token_list=[]
            for j in range(prv_id, cur_id):
                pred_token_list.append(predTokens[j])
            
            preds.append({
                'segment': prv_segment,
                'word_ids': pred_token_list
            })
            if i!=num_tokens:
                prv_segment = predSegment[i]
                prv_id=cur_id
    return preds

In [ ]:
def prediction(model, val_dataloader):
    model.eval()
    predvalues = []
    for inputs in val_dataloader:
        input_ids = inputs['input_ids']
        attn_mask = inputs['attn_mask']
        word_ids = inputs['word_ids']
        seq_len = inputs['seq_len']
        rpercentile = inputs['rpercentile']
        batch_max_seqlen = torch.max(seq_len).item()
        
        input_ids = input_ids[:, :batch_max_seqlen].to(device)
        attn_mask = attn_mask[:, :batch_max_seqlen].to(device)
        rpercentile = rpercentile[:, :batch_max_seqlen].to(device)
        with torch.no_grad():
            yhat = model(input_ids, attn_mask, rpercentile).softmax(dim=-1).argmax(dim=-1).cpu()
        
        bsize = attn_mask.shape[0]
        for i in range(bsize):
            yhati = yhat[i].numpy()
            word_ids_i = word_ids[i].numpy()
            
            pred_tokens = postprocess(yhati, word_ids_i)
            for token in pred_tokens:
                token['word_ids'] = [str(x) for x in token['word_ids']]
                token['word_ids'] = ' '.join(token['word_ids'])
            predvalues.append(pred_tokens)
    return predvalues

In [ ]:
test_df['predvalues'] = prediction(model, val_dataloader)
test_df.head()

In [ ]:
submission_data=[]
for index, row in test_df.iterrows():
    predvalues = row.predvalues
    
    for pred in predvalues:
        if len(pred['word_ids'].split()) == 1:
            continue
            
        submission_data.append({
            'id': row.id,
            'class': pred['segment'],
            'predictionstring': pred['word_ids']
        })
submission_df = pd.DataFrame.from_dict(submission_data)
submission_df.head()

In [ ]:
submission_df.to_csv("submission.csv", index=False)